In [31]:
import pandas as pd
import re 
df = pd.read_csv('dados/teste.csv')
metade = len(df)/2
df_teste = df[:int(metade)]
df_treino = df[int(metade):]

In [32]:

def limpa_texto(df):
    pontuacao = [',', '.', ';', ':', '!', '?', '-', '(', ')', '[', ']', '{', '}', '"', '/', '\\', '|', '_', '+', '=', '*', '&', '@', '#', '$', '%', '^', '<','>','`','~']
    lista = []
    for palavras in df:
        lista.append(palavras)
    string = ' '.join(lista)
    if '<br /><br />' in string:
        string = string.replace('<br /><br />', ' ')
    for pontos in pontuacao:
        if pontos in string:
            string = string.replace(f'{pontos}','')
    lista = re.split(r'\s+|\W+', string.lower())
    lista = [palavra for palavra in lista if palavra]
    return lista

def limpa_texto_review(df):
    pontuacao = [',', '.', ';', ':', '!', '?', '-', '(', ')', '[', ']', '{', '}', '"', '/', '\\', '|', '_', '+', '=', '*', '&', '@', '#', '$', '%', '^', '<','>','`','~']
    lista = df.split(' ') 
    string = ' '.join(lista)
    if '<br /><br />' in string:
        string = string.replace('<br /><br />', ' ')
    for pontos in pontuacao:
        if pontos in string:
            string = string.replace(f'{pontos}','')
    lista = re.split(r'\s+|\W+', string.lower())
    lista = [palavra for palavra in lista if palavra]
    return lista

lista_total = limpa_texto(df_teste['review'])

filtro_positivo = df_teste['sentiment'] == 'positive'
df_positivo = df_teste[filtro_positivo]
lista_positivo = limpa_texto(df_positivo['review'])

filtro_negativo = df_teste['sentiment'] == 'negative'
df_negativo = df_teste[filtro_negativo]
lista_negativo = limpa_texto(df_negativo['review'])

KeyboardInterrupt: 

In [ ]:
dic_contagem_total = {}
for i in range(len(df_teste)):
    lista_arrumada = limpa_texto_review(df_teste.iloc[i]['review'])
    lista_arrumada_unique = list(set(lista_arrumada))

    for palavras in lista_arrumada_unique:    
        if palavras not in dic_contagem_total:
            dic_contagem_total[palavras] = 1
        else:
            dic_contagem_total[palavras] += 1

dic_probabilidades_total = {}
for palavras, contagem in dic_contagem_total.items():
    dic_probabilidades_total[palavras] = contagem/len(df_teste)

In [ ]:
dic_contagem_positivo = {}
for i in range(len(df_positivo)):
    lista_arrumada = limpa_texto_review(df_positivo.iloc[i]['review'])
    lista_arrumada_unique = list(set(lista_arrumada))

    for palavras in lista_arrumada_unique:    
        if palavras not in dic_contagem_positivo:
            dic_contagem_positivo[palavras] = 1
        else:
            dic_contagem_positivo[palavras] += 1

dic_probabilidades_positivo = {}
for palavras, contagem in dic_contagem_positivo.items():
    dic_probabilidades_positivo[palavras] = contagem/len(df_positivo)

dic_probabilidades_barrada_positivo = {}
for palavra, probabilidade in dic_probabilidades_positivo.items():
    dic_probabilidades_barrada_positivo[palavra] = 1-probabilidade

for palavras in lista_negativo:
    if palavras not in dic_probabilidades_barrada_positivo:
        dic_probabilidades_barrada_positivo[palavras] = 1

In [ ]:
dic_contagem_negativo = {}
for i in range(len(df_negativo)):
    lista_arrumada = limpa_texto_review(df_negativo.iloc[i]['review'])
    lista_arrumada_unique = list(set(lista_arrumada))

    for palavras in lista_arrumada_unique:    
        if palavras not in dic_contagem_negativo:
            dic_contagem_negativo[palavras] = 1
        else:
            dic_contagem_negativo[palavras] += 1

dic_probabilidades_negativo = {}
for palavras, contagem in dic_contagem_negativo.items():
    dic_probabilidades_negativo[palavras] = contagem/len(df_negativo)

dic_probabilidades_barrada_negativo = {}
for palavra, probabilidade in dic_probabilidades_negativo.items():
    dic_probabilidades_barrada_negativo[palavra] = 1-probabilidade

for palavras in lista_positivo:
    if palavras not in dic_probabilidades_barrada_negativo:
        dic_probabilidades_barrada_negativo[palavras] = 1

In [ ]:
import math 
indice_acerto = 0

dic={}
for i in range(len(df_teste)):
    p_negativa, p_positiva = 0, 0
    negativo_confirmado,positivo_confirmado = False, False
    lista_arrumada = limpa_texto_review(df_teste.iloc[i]['review'])
    
    for palavras in lista_arrumada:  
        if palavras in dic_probabilidades_positivo:
            p_positiva += math.log10(dic_probabilidades_positivo[palavras]+0.001)
        else:
            negativo_confirmado = True
        if palavras in dic_probabilidades_negativo:
            p_negativa += math.log10(dic_probabilidades_negativo[palavras]+0.001)            
        else:
            positivo_confirmado = True
    
    for palavras2 in lista_total:    
        if palavras2 not in lista_arrumada:
            p_positiva += math.log10(dic_probabilidades_barrada_positivo[palavras2]+0.001)
        if palavras2 not in lista_arrumada:
            p_negativa += math.log10(dic_probabilidades_barrada_negativo[palavras2]+0.001)

    if positivo_confirmado:
        dic[i] = 'positive'
    elif negativo_confirmado:        
        dic[i] = 'negative'
    else:   
        if p_positiva > p_negativa:
            dic[i] = 'positive'
        else:
            dic[i] = 'negative'
for i in dic:
    if dic[i] == df_teste.iloc[i]['sentiment']:
        indice_acerto += 1

print(indice_acerto)
print(dic)

50
{0: 'positive', 1: 'positive', 2: 'positive', 3: 'negative', 4: 'positive', 5: 'positive', 6: 'positive', 7: 'negative', 8: 'negative', 9: 'positive', 10: 'negative', 11: 'negative', 12: 'negative', 13: 'negative', 14: 'positive', 15: 'negative', 16: 'positive', 17: 'negative', 18: 'positive', 19: 'negative', 20: 'positive', 21: 'negative', 22: 'positive', 23: 'negative', 24: 'negative', 25: 'positive', 26: 'positive', 27: 'negative', 28: 'negative', 29: 'positive', 30: 'positive', 31: 'positive', 32: 'negative', 33: 'positive', 34: 'negative', 35: 'negative', 36: 'negative', 37: 'negative', 38: 'positive', 39: 'negative', 40: 'negative', 41: 'positive', 42: 'negative', 43: 'negative', 44: 'positive', 45: 'positive', 46: 'negative', 47: 'negative', 48: 'positive', 49: 'negative'}
